In [1]:
from google.adk.agents import LlmAgent
from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmResponse, LlmRequest
from google.adk.runners import Runner
from typing import Optional
from google.genai import types 
from google.adk.sessions import InMemorySessionService

In [2]:
def my_before_model_logic(
    callback_context: CallbackContext, llm_request: LlmRequest) -> Optional[LlmResponse]:
    print(f"Callback running before model call for agent: {callback_context.agent_name}")
    return None 

In [3]:
def my_after_model_logic(
    callback_context: CallbackContext, llm_response: LlmResponse) -> Optional[LlmResponse]:
    print(f"Callback running after model call for agent: {callback_context.agent_name}")
    return None 

In [4]:
my_agent = LlmAgent(
    name="MyCallbackAgent",
    model="gemini-2.5-flash", 
    instruction="Be helpful.",
    before_model_callback=my_before_model_logic, 
    after_model_callback=my_after_model_logic
)

In [5]:
APP_NAME = "agents"
USER_ID = "user_1"
SESSION_ID = "session_001"

In [6]:
async def setup_session_and_runner():
    session_service = InMemorySessionService()
    session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
    runner = Runner(agent=my_agent, app_name=APP_NAME, session_service=session_service)
    return session, runner

In [7]:
async def call_agent_async(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    session, runner = await setup_session_and_runner()
    events = runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

    async for event in events:
        if event.is_final_response():
            final_response = event.content.parts[0].text
            print("Agent Response: ", final_response)

In [8]:
await call_agent_async("What's the capital of USA?")

Callback running before model call for agent: MyCallbackAgent
Callback running after model call for agent: MyCallbackAgent
Agent Response:  The capital of the USA is Washington, D.C.
